## The Perfect Fit
**A container problem solved.**

The containerstore.com has a large selection of containers, but does not provide a search option to specify the size of the container.  The Container class allows you to scrape the all the current storage data from the container store website storing the name, dimensions, image, and url for each piece.  There are functions to reorganize the data.  Then you can search the containers for the type, size, and/or price that you would like.


In [82]:
import scrapy
import json
from pprint import pprint
import subprocess
from fractions import Fraction
from collections import Counter 
import datetime


In [83]:
class Container:
    def __init__(self):
        pass
    
    def update_json(self, scrapy_name='container', json_file_name='container.json'):
        subprocess.run(["rm",json_file_name])
        subprocess.run(["scrapy", "crawl", scrapy_name, "-o", json_file_name])
        print('updated on',datetime.date.today())

    def load_json(self, json_file_name='container.json'):
        with open(json_file_name) as data_file:    
            self.data = json.load(data_file)
    
    
    def create_new_dimensions(self):
        
        # loop through the container links
        for j in range (0,len(self.data)):
            self.data[j]['new dimensions']=[]
            # many container links have containers available in several different sizes, so we roam through each one
            for k in range(0,len(self.data[j]['dimensions'])):
                dim_no_whitespace = self.data[j]['dimensions'][k].strip()
                if (dim_no_whitespace != ""):
                    mydim = self.data[j]['dimensions'][k].split('x')
                    newdim = []
                    # each dimension of the container needs to be converted to a float and added to 'new dimensions'
                    for i in range (0,len(mydim)):
                        mysplit = mydim[i].split('sq.')[0]
                        if '-' in mysplit:
                            this_split = mysplit.split('-')
                            num =0
                            for p in this_split:
                                num += float(Fraction(p.split('"')[0]))
                        else:
                            num = float(Fraction(mysplit.split('"')[0]))
                        # if the entry notes that it is square or a diameter, then the dimension needs to be inserted in two spots            
                        double=int(('sq.' in mydim[i]) or ('diam.' in mydim[i]))
                        for _ in range(0,double+1):
                            newdim.append(num)
                    self.data[j]['new dimensions'].append(newdim)

    def organize_new_dimensions(self):
        self.dimensions = [[],[],[],[]] #stores zero, one, two, three dimensions
        for j in range(0,len(self.data)):
            if((self.data[j]['new dimensions'])==[]):
                self.dimensions[0].append([self.data[j],k]) 
            num_of_containers  = len(self.data[j]['new dimensions'])
            for k in range(0,num_of_containers):
                num_dim = len(self.data[j]['new dimensions'][k])
                
                
                #save the container and the spot that has these particular dimensions
                self.dimensions[num_dim].append([self.data[j],k]) 

                

    def create_category(self):
        for d in self.data:
            category = d['url'].split('/')[5]
            d['category']=category
        self.c = Counter()
        for d in self.data:
            self.c[d['category']] += 1
#         return (self.c)
        

    #  The will_it_fit function will be given 6 values, 2 for each dimension (a lower and upper limit).  
    #  It returns the possible containers.
    def the_perfect_fit(self,f1a,f1b,f2a,f2b,f3a,f3b):

        possibles = []
        for j in range(0,len(self.data)):
            num_of_containers  = len(self.data[j]['new dimensions'])
            for k in range(0,num_of_containers):
                if (len(self.data[j]['new dimensions'][k])==3):

                    if ((f1a<self.data[j]['new dimensions'][k][0]<f1b) and (f2a<self.data[j]['new dimensions'][k][1]<f2b) 
                        and (f3a<self.data[j]['new dimensions'][k][2]<f3b)):

                        possibles.append([self.data[j],k])

        return possibles
    

In [84]:
my_c = Container()
# my_c.update_json() 
my_c.load_json('container.json')
print('')
print('We found',len(my_c.data), 'total containers.')    
print('')

my_c.create_category()
print(my_c.c.most_common())

updated on 2017-07-28

We found 658 total containers.

[('decorative-bins-baskets', 210), ('stacking-storage', 111), ('storage-drawers', 72), ('storage-bags-totes', 55), ('plastic-bins-baskets', 52), ('modular-storage', 46), ('serving-trays', 41), ('garage-storage-boxes', 23), ('storage-benches-seats', 22), ('smart-store', 16), ('trunks', 10)]


In [85]:
my_c.create_new_dimensions()
my_c.organize_new_dimensions()

The container that I am looking for should be in the range of 6-8.25" x 9-12" x 5-7"

In [175]:
f1a = 6.
f1b = 8.25
f2a = 9.
f2b = 12.
f3a = 5.
f3b = 7.

# f1a = 12.
# f1b = 15.
# f2a = 12.
# f2b = 15.
# f3a = 12.
# f3b = 15.


possibles = my_c.the_perfect_fit(f1a,f1b,f2a,f2b,f3a,f3b)

print('There are',len(possibles), 'containers that will fit your space:')
print('')
for c in possibles:
    print('')
    print(c[0]['title'], ", '"+str(c[0]['category'])+ "'")
    print(c[0]['url'])
    print(c[0]['dimensions'][c[1]])

    print('$' +str(c[0]['price'][c[1]]))



There are 8 containers that will fit your space:


White Handled Storage Baskets , 'plastic-bins-baskets'
https://www.containerstore.com/s/storage/plastic-bins-baskets/white-handled-storage-baskets/12d?productId=10036528
6-1/4" x 11" x 5-1/8" h
$3.99

Clear Handled Storage Baskets , 'plastic-bins-baskets'
https://www.containerstore.com/s/storage/plastic-bins-baskets/clear-handled-storage-baskets/12d?productId=10022155
6-1/4" x 11" x 5-1/8" h
$3.99

Linus Storage Binz , 'plastic-bins-baskets'
https://www.containerstore.com/s/storage/plastic-bins-baskets/linus-storage-binz/12d?productId=11000989
8" x 11-1/4" x 6" h
$16.99

Mondrian Storage Boxes with Lids , 'plastic-bins-baskets'
https://www.containerstore.com/s/storage/plastic-bins-baskets/mondrian-storage-boxes-with-lids/12d?productId=10036979
7" x 10-5/8" x 5-3/8" h
$6.99

Small Rectangular Hogla Storage Bin with Handles , 'decorative-bins-baskets'
https://www.containerstore.com/s/storage/decorative-bins-baskets/small-rectangular-hogl

In [179]:

no_dim = my_c.find_no_dimensions()

print('')
print('We found',len(no_dim), 'containers without any dimensions given.')    
print('')

for n in no_dim:
    print(n['title'],n['url'])


ld = my_c.low_dimensions()
print('')
print('These are', len(ld[0]), 'results that only give 1 dimension. These are all dividers.')    
print('')
for o in ld[0]:
        print(o[0]['title'],o[0]['new dimensions'][o[1]],o[0]['url'])
print('')
print('These are', len(ld[1]), 'results that only give 2 dimensions. Some are dividers, others are plastic bags, neither are interesting to us for now.  Unfortunately there are some stackable containers that only give the base dimensions, so we would like to find and include these in our searches.')    
print('')     
for o in ld[1]:
        print(o[0]['title'],o[0]['new dimensions'][o[1]],o[0]['url'])      


We found 9 containers without any dimensions given.

White Compact Plastic Bins 4-Pack with Grey Lids https://www.containerstore.com/s/storage/modular-storage/white-compact-plastic-bins-4-pack-with-grey-lids/12d?productId=11006520
White Compact Plastic Bins 4-Pack with White Lids https://www.containerstore.com/s/storage/plastic-bins-baskets/white-compact-plastic-bins-4-pack-with-white-lids/12d?productId=11006211
White Compact Plastic Bins 4-Pack with White Lids https://www.containerstore.com/s/storage/modular-storage/white-compact-plastic-bins-4-pack-with-white-lids/12d?productId=11006211
White Compact Plastic Bins 4-Pack with Grey Lids https://www.containerstore.com/s/storage/plastic-bins-baskets/white-compact-plastic-bins-4-pack-with-grey-lids/12d?productId=11006520
Assorted Reclosable Bags Pkg/40 https://www.containerstore.com/s/storage/storage-bags-totes/assorted-reclosable-bags/12d?productId=10030813
Reclosable Bags https://www.containerstore.com/s/storage/storage-bags-totes/recl